# the import of libraris

In [220]:

import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re 
import time 

In [214]:
import requests

url = "https://www.bayut.om/_next/data/8601/en/oman/properties-for-sale.json"

headers = {
    'User-Agent': 'Mozilla/5.0'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()

    # Listings are buried in JSON:
    properties = data['pageProps']['props']['pageProps']['listingProperties']

    for item in properties:
        title = item.get('title', 'N/A')
        price = item.get('price', 'N/A')
        location = item.get('location', [])
        location_str = ", ".join([loc.get('name') for loc in location]) if location else 'N/A'

        print("Title:", title)
        print("Price:", price)
        print("Location:", location_str)
        print("-" * 40)
else:
    print("Failed to fetch listings. Status code:", response.status_code)


Failed to fetch listings. Status code: 404


In [225]:
import requests
from bs4 import BeautifulSoup
import json
import re

url = 'https://www.bayut.om/en/oman/properties-for-sale'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all article elements with property listings
listings = soup.find_all('article', class_="fbc619bc")

print(f"Found {len(listings)} property listings")

if response.status_code == 200:
    for item in listings:
        # Try to extract data from JSON-LD first (most reliable)
        json_data = item.find('script', type='application/ld+json')
        if json_data:
            try:
                data = json.loads(json_data.string)
                title = data.get('name', 'N/A')
                price = item.select_one('span[aria-label="Price"]')
                price_text = price.text if price else 'N/A'
                location = data.get('address', {}).get('addressLocality', 'N/A')
                if not location or location == 'N/A':
                    # Fallback to looking for location in the HTML
                    location_elem = item.select_one('div[aria-label="Location"]')
                    location = location_elem.text if location_elem else 'N/A'
                
                print("Title:", title)
                print("Price:", price_text)
                print("Location:", location)
                print("-" * 40)
                continue
            except (json.JSONDecodeError, AttributeError):
                pass  # If JSON extraction fails, fall back to HTML parsing
        
        # Fallback to HTML parsing
        title_elem = item.find(['h2', 'h3'])
        price_elem = item.select_one('span[aria-label="Price"]')
        location_elem = item.select_one('div[aria-label="Location"]')
        
        title = title_elem.text.strip() if title_elem else "N/A"
        price = price_elem.text.strip() if price_elem else "N/A"
        location = location_elem.text.strip() if location_elem else "N/A"
        
        print("Title:", title)
        print("Price:", price)
        print("Location:", location)
        print("-" * 40)
else:
    print("Failed to fetch the page. Status code:", response.status_code)

Found 24 property listings
Title: 5 Bedrooms Villa For Sale in Omani City
Price: 1,500
Location: Qantab
----------------------------------------
Title: 3 Bedrooms Apartment For Sale Mahut, Al Wusta
Price: 1,200
Location: Mahut
----------------------------------------
Title: 5 Bedrooms Villa For Sale in Al Sifah, Muscat
Price: 1,500
Location: Al Sifah
----------------------------------------
Title: 4 Bedrooms Villa For Sale in Hamra, Al Dakhiliya
Price: 1,000
Location: Hamra
----------------------------------------
Title: 3 Bedrooms Apartment For Sale in Oman
Price: 120,000
Location: Al Sinainah
----------------------------------------
Title: 2 Bedrooms Apartment For Sale in Oman
Price: 100,000
Location: Al Duqum
----------------------------------------
Title: 3 Bedrooms Apartment For Sale in Oman
Price: 220,000
Location: Al Duqum
----------------------------------------
Title: 2 Bedrooms Apartment For Sale in Oman
Price: 22,000
Location: Muqshin
----------------------------------------

In [233]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
from IPython.display import display

# Base URL for the property listings
base_url = 'https://www.bayut.om/en/oman/properties-for-sale'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Create lists to store the data
all_titles = []
all_prices = []
all_locations = []
all_urls = []

# Start with page 1 and continue until no more listings are found
page = 1
continue_scraping = True

print("Starting to scrape data. This may take some time...")

while continue_scraping:
    # Construct the URL for each page
    if page == 1:
        url = base_url
    else:
        url = f"{base_url}/page-{page}"
    
    print(f"Scraping page {page}: {url}")
    
    # Add a delay between requests to avoid being blocked
    if page > 1:
        time.sleep(3)  # 3 seconds delay between pages
    
    try:
        response = requests.get(url, headers=headers)
        
        # Check if the page was successfully retrieved
        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all article elements with property listings
        listings = soup.find_all('article', class_="fbc619bc")
        
        print(f"Found {len(listings)} property listings on page {page}")
        
        if not listings:
            print("No more listings found. Stopping.")
            break
        
        for item in listings:
            # Try to extract data from JSON-LD first (most reliable)
            json_data = item.find('script', type='application/ld+json')
            
            title = "N/A"
            price = "N/A"
            location = "N/A"
            property_url = "N/A"
            
            if json_data:
                try:
                    data = json.loads(json_data.string)
                    title = data.get('name', 'N/A')
                    property_url = data.get('url', 'N/A')
                    
                    # Get location from JSON if available
                    if 'address' in data and 'addressLocality' in data['address']:
                        location = data['address']['addressLocality']
                except (json.JSONDecodeError, AttributeError):
                    pass
            
            # Get price from HTML (usually not in JSON-LD)
            price_elem = item.select_one('span[aria-label="Price"]')
            if price_elem:
                price = price_elem.text.strip()
            
            # If location not found in JSON, try HTML
            if location == "N/A":
                location_elem = item.select_one('div[aria-label="Location"]')
                if location_elem:
                    location = location_elem.text.strip()
            
            # If title not found in JSON, try HTML
            if title == "N/A":
                title_elem = item.find(['h2', 'h3'])
                if title_elem:
                    title = title_elem.text.strip()
            
            # If URL not found in JSON, try HTML
            if property_url == "N/A":
                url_elem = item.find('a')
                if url_elem and 'href' in url_elem.attrs:
                    property_url = 'https://www.bayut.om' + url_elem['href'] if url_elem['href'].startswith('/') else url_elem['href']
            
            # Append data to lists
            all_titles.append(title)
            all_prices.append(price)
            all_locations.append(location)
            all_urls.append(property_url)
        
        # Check if there's a "next page" link
        next_page = soup.select_one('a[title="Next"]')
        if not next_page:
            print("No 'Next' button found. Reached the last page.")
            continue_scraping = False
        else:
            page += 1
            
            # Save intermediate results every 5 pages
            if page % 5 == 0:
                print(f"Completed {page-1} pages. {len(all_titles)} properties collected so far.")
    
    except Exception as e:
        print(f"Error occurred on page {page}: {str(e)}")
        break

print(f"\nScraping completed! Collected data from {page} pages with a total of {len(all_titles)} properties.")

# Create a DataFrame with all the data
final_df = pd.DataFrame({
    'Title': all_titles,
    'Price': all_prices,
    'Location': all_locations,
    'URL': all_urls
})

# Save to CSV
final_df.to_csv('bayut_properties_all.csv', index=False)
print(f"Data saved to bayut_properties_all.csv")

# Display the final DataFrame
print("\nDisplaying all properties in a single table:")
display(final_df)

Starting to scrape data. This may take some time...
Scraping page 1: https://www.bayut.om/en/oman/properties-for-sale
Found 24 property listings on page 1
Scraping page 2: https://www.bayut.om/en/oman/properties-for-sale/page-2
Found 24 property listings on page 2
Scraping page 3: https://www.bayut.om/en/oman/properties-for-sale/page-3
Found 24 property listings on page 3
Scraping page 4: https://www.bayut.om/en/oman/properties-for-sale/page-4
Found 24 property listings on page 4
Completed 4 pages. 96 properties collected so far.
Scraping page 5: https://www.bayut.om/en/oman/properties-for-sale/page-5
Found 24 property listings on page 5
Scraping page 6: https://www.bayut.om/en/oman/properties-for-sale/page-6
Found 24 property listings on page 6
Scraping page 7: https://www.bayut.om/en/oman/properties-for-sale/page-7
Found 24 property listings on page 7
Scraping page 8: https://www.bayut.om/en/oman/properties-for-sale/page-8
Found 24 property listings on page 8
Scraping page 9: https:/

,Title,Price,Location,URL
0,5 Bedrooms Villa For Sale in Omani City,"1,500",Qantab,https://www.bayut.om/en/property/details-50002...
1,"3 Bedrooms Apartment For Sale Mahut, Al Wusta","1,200",Mahut,https://www.bayut.om/en/property/details-50002...
2,"5 Bedrooms Villa For Sale in Al Sifah, Muscat","1,500",Al Sifah,https://www.bayut.om/en/property/details-50002...
3,"4 Bedrooms Villa For Sale in Hamra, Al Dakhiliya","1,000",Hamra,https://www.bayut.om/en/property/details-50002...
4,3 Bedrooms Apartment For Sale in Oman,"120,000",Al Sinainah,https://www.bayut.om/en/property/details-50002...
...,...,...,...,...
883,"2 Bedrooms Apartment For Sale in Muscat Hills,...","100,000",Muscat Hills,https://www.bayut.om/en/property/details-12986...
884,"6 Bedroom Villa For Sale in Al Mawaleh, Muscat","146,000",Al Mawaleh,https://www.bayut.om/en/property/details-12980...
885,"Residential Land For Sale in Barka, Al Batinah","120,000",Barka,https://www.bayut.om/en/property/details-12992...
886,1 Bedroom Apartment For Sale in The Wave (Almo...,"79,500",The Wave (Almouj),https://www.bayut.om/en/property/details-12999...


In [234]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
from IPython.display import display

# Base URL for the property listings
base_url = 'https://www.bayut.om/en/oman/properties-for-sale'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Create lists to store the data
all_titles = []
all_prices = []
all_locations = []
all_urls = []

# Start with page 1 and continue until no more listings are found
page = 1
continue_scraping = True

print("Starting to scrape data. This may take some time...")

while continue_scraping:
    # Construct the URL for each page
    if page == 1:
        url = base_url
    else:
        url = f"{base_url}/page-{page}"
    
    print(f"Scraping page {page}: {url}")
    
    # Add a delay between requests to avoid being blocked
    if page > 1:
        time.sleep(3)  # 3 seconds delay between pages
    
    try:
        response = requests.get(url, headers=headers)
        
        # Check if the page was successfully retrieved
        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all article elements with property listings
        listings = soup.find_all('article', class_="fbc619bc")
        
        print(f"Found {len(listings)} property listings on page {page}")
        
        if not listings:
            print("No more listings found. Stopping.")
            break
        
        for item in listings:
            # Try to extract data from JSON-LD first (most reliable)
            json_data = item.find('script', type='application/ld+json')
            
            title = "N/A"
            price = "N/A"
            location = "N/A"
            property_url = "N/A"
            
            if json_data:
                try:
                    data = json.loads(json_data.string)
                    title = data.get('name', 'N/A')
                    property_url = data.get('url', 'N/A')
                    
                    # Get location from JSON if available
                    if 'address' in data and 'addressLocality' in data['address']:
                        location = data['address']['addressLocality']
                except (json.JSONDecodeError, AttributeError):
                    pass
            
            # Get price from HTML (usually not in JSON-LD)
            price_elem = item.select_one('span[aria-label="Price"]')
            if price_elem:
                price = price_elem.text.strip()
            
            # If location not found in JSON, try HTML
            if location == "N/A":
                location_elem = item.select_one('div[aria-label="Location"]')
                if location_elem:
                    location = location_elem.text.strip()
            
            # If title not found in JSON, try HTML
            if title == "N/A":
                title_elem = item.find(['h2', 'h3'])
                if title_elem:
                    title = title_elem.text.strip()
            
            # If URL not found in JSON, try HTML
            if property_url == "N/A":
                url_elem = item.find('a')
                if url_elem and 'href' in url_elem.attrs:
                    property_url = 'https://www.bayut.om' + url_elem['href'] if url_elem['href'].startswith('/') else url_elem['href']
            
            # Append data to lists
            all_titles.append(title)
            all_prices.append(price)
            all_locations.append(location)
            all_urls.append(property_url)
        
        # Check if there's a "next page" link
        next_page = soup.select_one('a[title="Next"]')
        if not next_page:
            print("No 'Next' button found. Reached the last page.")
            continue_scraping = False
        else:
            page += 1
            
            # Save intermediate results every 5 pages
            if page % 5 == 0:
                print(f"Completed {page-1} pages. {len(all_titles)} properties collected so far.")
    
    except Exception as e:
        print(f"Error occurred on page {page}: {str(e)}")
        break

print(f"\nScraping completed! Collected data from {page} pages with a total of {len(all_titles)} properties.")

# Create a DataFrame with all the data
final_df = pd.DataFrame({
    'Title': all_titles,
    'Price': all_prices,
    'Location': all_locations,
    'URL': all_urls
})

# Save to CSV
final_df.to_csv('bayut_properties_all.csv', index=False)
print(f"Data saved to bayut_properties_all.csv")

# Display the final DataFrame
print("\nDisplaying all properties in a single table:")
display(final_df)

Starting to scrape data. This may take some time...
Scraping page 1: https://www.bayut.om/en/oman/properties-for-sale
Found 24 property listings on page 1
Scraping page 2: https://www.bayut.om/en/oman/properties-for-sale/page-2
Found 24 property listings on page 2
Scraping page 3: https://www.bayut.om/en/oman/properties-for-sale/page-3
Found 24 property listings on page 3
Scraping page 4: https://www.bayut.om/en/oman/properties-for-sale/page-4
Found 24 property listings on page 4
Completed 4 pages. 96 properties collected so far.
Scraping page 5: https://www.bayut.om/en/oman/properties-for-sale/page-5
Found 24 property listings on page 5
Scraping page 6: https://www.bayut.om/en/oman/properties-for-sale/page-6
Found 24 property listings on page 6
Scraping page 7: https://www.bayut.om/en/oman/properties-for-sale/page-7
Found 24 property listings on page 7
Scraping page 8: https://www.bayut.om/en/oman/properties-for-sale/page-8
Found 24 property listings on page 8
Scraping page 9: https:/

,Title,Price,Location,URL
0,5 Bedrooms Villa For Sale in Omani City,"1,500",Qantab,https://www.bayut.om/en/property/details-50002...
1,"3 Bedrooms Apartment For Sale Mahut, Al Wusta","1,200",Mahut,https://www.bayut.om/en/property/details-50002...
2,"5 Bedrooms Villa For Sale in Al Sifah, Muscat","1,500",Al Sifah,https://www.bayut.om/en/property/details-50002...
3,"4 Bedrooms Villa For Sale in Hamra, Al Dakhiliya","1,000",Hamra,https://www.bayut.om/en/property/details-50002...
4,3 Bedrooms Apartment For Sale in Oman,"120,000",Al Sinainah,https://www.bayut.om/en/property/details-50002...
...,...,...,...,...
619,"4 Bedrooms Villa For Sale Muscat Hills, Muscat","210,000",Muscat Hills,https://www.bayut.om/en/property/details-13004...
620,"4 Bedrooms Villa For Sale in Muscat Hills, Muscat","310,000",Muscat Hills,https://www.bayut.om/en/property/details-13004...
621,2 Bedrooms Apartment For Sale Muscat (The Wave...,"210,000",The Wave (Almouj),https://www.bayut.om/en/property/details-13020...
622,"1 Bedroom Apartment For Sale Salalah, Dhofar","55,000",Salalah,https://www.bayut.om/en/property/details-13003...


In [235]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
from IPython.display import display

# Base URL for the property listings
base_url = 'https://www.bayut.om/en/oman/properties-for-sale'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Create lists to store the data
all_titles = []
all_prices = []
all_locations = []
all_urls = []

# Start with page 1 and continue until no more listings are found
page = 1
continue_scraping = True

print("Starting to scrape data. This may take some time...")

while continue_scraping:
    # Construct the URL for each page
    if page == 1:
        url = base_url
    else:
        url = f"{base_url}/page-{page}"
    
    print(f"Scraping page {page}: {url}")
    
    # Add a delay between requests to avoid being blocked
    if page > 1:
        time.sleep(3)  # 3 seconds delay between pages
    
    try:
        response = requests.get(url, headers=headers)
        
        # Check if the page was successfully retrieved
        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all article elements with property listings
        listings = soup.find_all('article', class_="fbc619bc")
        
        print(f"Found {len(listings)} property listings on page {page}")
        
        if not listings:
            print("No more listings found. Stopping.")
            break
        
        for item in listings:
            # Try to extract data from JSON-LD first (most reliable)
            json_data = item.find('script', type='application/ld+json')
            
            title = "N/A"
            price = "N/A"
            location = "N/A"
            property_url = "N/A"
            
            if json_data:
                try:
                    data = json.loads(json_data.string)
                    title = data.get('name', 'N/A')
                    property_url = data.get('url', 'N/A')
                    
                    # Get location from JSON if available
                    if 'address' in data and 'addressLocality' in data['address']:
                        location = data['address']['addressLocality']
                except (json.JSONDecodeError, AttributeError):
                    pass
            
            # Get price from HTML (usually not in JSON-LD)
            price_elem = item.select_one('span[aria-label="Price"]')
            if price_elem:
                price = price_elem.text.strip()
            
            # If location not found in JSON, try HTML
            if location == "N/A":
                location_elem = item.select_one('div[aria-label="Location"]')
                if location_elem:
                    location = location_elem.text.strip()
            
            # If title not found in JSON, try HTML
            if title == "N/A":
                title_elem = item.find(['h2', 'h3'])
                if title_elem:
                    title = title_elem.text.strip()
            
            # If URL not found in JSON, try HTML
            if property_url == "N/A":
                url_elem = item.find('a')
                if url_elem and 'href' in url_elem.attrs:
                    property_url = 'https://www.bayut.om' + url_elem['href'] if url_elem['href'].startswith('/') else url_elem['href']
            
            # Append data to lists
            all_titles.append(title)
            all_prices.append(price)
            all_locations.append(location)
            all_urls.append(property_url)
        
        # Check if there's a "next page" link
        next_page = soup.select_one('a[title="Next"]')
        if not next_page:
            print("No 'Next' button found. Reached the last page.")
            continue_scraping = False
        else:
            page += 1
            
            # Save intermediate results every 5 pages
            if page % 5 == 0:
                print(f"Completed {page-1} pages. {len(all_titles)} properties collected so far.")
    
    except Exception as e:
        print(f"Error occurred on page {page}: {str(e)}")
        break

print(f"\nScraping completed! Collected data from {page} pages with a total of {len(all_titles)} properties.")

# Create a DataFrame with all the data
final_df = pd.DataFrame({
    'Title': all_titles,
    'Price': all_prices,
    'Location': all_locations,
    'URL': all_urls
})

# Save to CSV
final_df.to_csv('bayut_properties_all.csv', index=False)
print(f"Data saved to bayut_properties_all.csv")

# Display the final DataFrame
print("\nDisplaying all properties in a single table:")
display(final_df)

Starting to scrape data. This may take some time...
Scraping page 1: https://www.bayut.om/en/oman/properties-for-sale
Found 24 property listings on page 1
Scraping page 2: https://www.bayut.om/en/oman/properties-for-sale/page-2
Found 24 property listings on page 2
Scraping page 3: https://www.bayut.om/en/oman/properties-for-sale/page-3
Found 24 property listings on page 3
Scraping page 4: https://www.bayut.om/en/oman/properties-for-sale/page-4
Found 24 property listings on page 4
Completed 4 pages. 96 properties collected so far.
Scraping page 5: https://www.bayut.om/en/oman/properties-for-sale/page-5
Found 24 property listings on page 5
Scraping page 6: https://www.bayut.om/en/oman/properties-for-sale/page-6
Found 24 property listings on page 6
Scraping page 7: https://www.bayut.om/en/oman/properties-for-sale/page-7
Found 24 property listings on page 7
Scraping page 8: https://www.bayut.om/en/oman/properties-for-sale/page-8
Found 24 property listings on page 8
Scraping page 9: https:/

,Title,Price,Location,URL
0,5 Bedrooms Villa For Sale in Omani City,"1,500",Qantab,https://www.bayut.om/en/property/details-50002...
1,"3 Bedrooms Apartment For Sale Mahut, Al Wusta","1,200",Mahut,https://www.bayut.om/en/property/details-50002...
2,"5 Bedrooms Villa For Sale in Al Sifah, Muscat","1,500",Al Sifah,https://www.bayut.om/en/property/details-50002...
3,"4 Bedrooms Villa For Sale in Hamra, Al Dakhiliya","1,000",Hamra,https://www.bayut.om/en/property/details-50002...
4,3 Bedrooms Apartment For Sale in Oman,"120,000",Al Sinainah,https://www.bayut.om/en/property/details-50002...
...,...,...,...,...
1803,"4 Bedrooms Villa For Sale in Haima, Al Wusta","1,000",Haima,https://www.bayut.om/en/property/details-50001...
1804,"1 Bedroom Apartment For Sale in Mahut, Al Wusta",600,Mahut,https://www.bayut.om/en/property/details-50001...
1805,Land For Sale - Residential Land - Al Dakhiliy...,"5,800",Nizwa,https://www.bayut.om/en/property/details-13008...
1806,"4 Bedrooms Villa For Sale Yiti, Muscat","491,410",Yiti,https://www.bayut.om/en/property/details-12991...


In [236]:
final_df

,Title,Price,Location,URL
0,5 Bedrooms Villa For Sale in Omani City,"1,500",Qantab,https://www.bayut.om/en/property/details-50002...
1,"3 Bedrooms Apartment For Sale Mahut, Al Wusta","1,200",Mahut,https://www.bayut.om/en/property/details-50002...
2,"5 Bedrooms Villa For Sale in Al Sifah, Muscat","1,500",Al Sifah,https://www.bayut.om/en/property/details-50002...
3,"4 Bedrooms Villa For Sale in Hamra, Al Dakhiliya","1,000",Hamra,https://www.bayut.om/en/property/details-50002...
4,3 Bedrooms Apartment For Sale in Oman,"120,000",Al Sinainah,https://www.bayut.om/en/property/details-50002...
...,...,...,...,...
1803,"4 Bedrooms Villa For Sale in Haima, Al Wusta","1,000",Haima,https://www.bayut.om/en/property/details-50001...
1804,"1 Bedroom Apartment For Sale in Mahut, Al Wusta",600,Mahut,https://www.bayut.om/en/property/details-50001...
1805,Land For Sale - Residential Land - Al Dakhiliy...,"5,800",Nizwa,https://www.bayut.om/en/property/details-13008...
1806,"4 Bedrooms Villa For Sale Yiti, Muscat","491,410",Yiti,https://www.bayut.om/en/property/details-12991...
